In [12]:
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import numpy as np



import pandas as pd
import os
import sys
import numpy as np
import seaborn as sns
import random
from tqdm import tqdm
import math

sys.path.append('../utils')
import helpers as h
import feature_engineering as f


try:
    import geopandas as gpd
except:
    ! pip install geopandas

try: 
    import scipy as sp 
except:
    ! pip install scipy

try: 
    from ortools.constraint_solver import routing_enums_pb2
    from ortools.constraint_solver import pywrapcp

except:
    ! pip install ortools


from polycluster import RoutePolygonManager
import plotly.graph_objects as go
from shapely.geometry import mapping

In [19]:
def cluster_points(points, n_clusters):
    """
    Clusters 2D points into exactly n_clusters, good for non-spherical, bunched-up groups.

    Args:
        points (array-like): List or array of 2D points [(x, y), ...].
        n_clusters (int): Number of clusters to find.

    Returns:
        np.ndarray: Array of cluster labels for each point.
    """
    points = np.array(points)

    model = AgglomerativeClustering(
        n_clusters=n_clusters,
        linkage='single',  # good for chaining tightly packed points
    )

    labels = model.fit_predict(points)
    return labels

In [5]:
ITM_DIR = os.path.join(os.getcwd(), '../data/intermediate')
request_df = pd.read_csv(os.path.join(ITM_DIR, 'clean_data.csv'))


C:\Users\Frede\AppData\Local\Temp\ipykernel_92796\3080534967.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  request_df = pd.read_csv(os.path.join(ITM_DIR, 'clean_data.csv'))


In [6]:
driven_df = request_df.query('request_day_route_is_last==True')

In [7]:
driven_df

,file_uuid,configurationName,id,lat,long,start_time,end_time,server_task_number,stop_order,dir,...,Date,Time,OptimizationRequestId,RouteId,TriggerType,ConfigurationName,NumberOfTasks,NumberOfTasksInInputPlan,request_day_route_order,request_day_route_is_last
4,34f409b3-9f92-4720-9926-48ca1cbf6f90,CreateSequence,388,0.572716,0.233306,2022-06-17T07:30:00,2022-06-17T23:59:00,2,1,0521_300-20220617,...,2022-06-17 00:00:00,1900-01-01 09:24:16.831000,34f409b3-9f92-4720-9926-48ca1cbf6f90,0521_300,FullOptimization,CreateSequence,2.0,0.0,3.0,True
5,34f409b3-9f92-4720-9926-48ca1cbf6f90,CreateSequence,390,0.565826,0.221868,2022-06-17T08:00:00,2022-06-17T17:00:00,1,2,0521_300-20220617,...,2022-06-17 00:00:00,1900-01-01 09:24:16.831000,34f409b3-9f92-4720-9926-48ca1cbf6f90,0521_300,FullOptimization,CreateSequence,2.0,0.0,3.0,True
6,cb9638db-2817-4ccf-b226-7775c01793a8,EstimateTime,64869,0.650488,0.296025,2022-05-30T00:00:00,2022-05-30T12:00:00,1,1,0521_301-20220530,...,2022-05-30 00:00:00,1900-01-01 06:45:38.424000,cb9638db-2817-4ccf-b226-7775c01793a8,0521_301,TimeCalculation,EstimateTime,154.0,154.0,1.0,True
7,cb9638db-2817-4ccf-b226-7775c01793a8,EstimateTime,64947,0.646361,0.297741,2022-05-30T00:00:00,2022-05-30T12:00:00,2,2,0521_301-20220530,...,2022-05-30 00:00:00,1900-01-01 06:45:38.424000,cb9638db-2817-4ccf-b226-7775c01793a8,0521_301,TimeCalculation,EstimateTime,154.0,154.0,1.0,True
8,cb9638db-2817-4ccf-b226-7775c01793a8,EstimateTime,64971,0.645787,0.298443,2022-05-30T00:00:00,2022-05-30T12:00:00,3,3,0521_301-20220530,...,2022-05-30 00:00:00,1900-01-01 06:45:38.424000,cb9638db-2817-4ccf-b226-7775c01793a8,0521_301,TimeCalculation,EstimateTime,154.0,154.0,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2120926,8a8e6d6b-045c-44bd-9085-cf2d5ec7a6d1,EstimateTime,63699,0.480072,0.301707,2022-06-22T07:30:00,2022-06-22T23:59:00,135,135,0521_O69-20220622,...,2022-06-22 00:00:00,1900-01-01 08:38:33.987000,8a8e6d6b-045c-44bd-9085-cf2d5ec7a6d1,0521_O69,TimeCalculation,EstimateTime,139.0,139.0,5.0,True
2120927,8a8e6d6b-045c-44bd-9085-cf2d5ec7a6d1,EstimateTime,63698,0.479680,0.301925,2022-06-22T07:30:00,2022-06-22T23:59:00,136,136,0521_O69-20220622,...,2022-06-22 00:00:00,1900-01-01 08:38:33.987000,8a8e6d6b-045c-44bd-9085-cf2d5ec7a6d1,0521_O69,TimeCalculation,EstimateTime,139.0,139.0,5.0,True
2120928,8a8e6d6b-045c-44bd-9085-cf2d5ec7a6d1,EstimateTime,63697,0.479792,0.300703,2022-06-22T07:30:00,2022-06-22T23:59:00,137,137,0521_O69-20220622,...,2022-06-22 00:00:00,1900-01-01 08:38:33.987000,8a8e6d6b-045c-44bd-9085-cf2d5ec7a6d1,0521_O69,TimeCalculation,EstimateTime,139.0,139.0,5.0,True
2120929,8a8e6d6b-045c-44bd-9085-cf2d5ec7a6d1,EstimateTime,63696,0.481527,0.300490,2022-06-22T07:30:00,2022-06-22T23:59:00,138,138,0521_O69-20220622,...,2022-06-22 00:00:00,1900-01-01 08:38:33.987000,8a8e6d6b-045c-44bd-9085-cf2d5ec7a6d1,0521_O69,TimeCalculation,EstimateTime,139.0,139.0,5.0,True


In [10]:
day = "2022-06-13"
day_df = driven_df.query('route_date==@day').copy()

points = list(zip(day_df['lat'], day_df['long']))
cars_available = day_df.route_id.nunique()

178

In [32]:
from sklearn.cluster import AgglomerativeClustering, KMeans
import numpy as np

def cluster_points(points, n_clusters, max_cluster_size=250):
    """
    Clusters 2D points into up to n_clusters. Ensures no cluster exceeds max_cluster_size.

    Args:
        points (array-like): List or array of 2D points [(x, y), ...].
        n_clusters (int): Desired number of clusters (initial clusters before splitting).
        max_cluster_size (int): Maximum number of points allowed per cluster.

    Returns:
        np.ndarray: Cluster labels for each point (0-indexed).
    """
    points = np.array(points)
    agg = AgglomerativeClustering(n_clusters=n_clusters, linkage='single')
    initial_labels = agg.fit_predict(points)

    final_labels = np.zeros(len(points), dtype=int)
    label_counter = 0

    for cluster_id in np.unique(initial_labels):
        cluster_indices = np.where(initial_labels == cluster_id)[0]
        cluster_size = len(cluster_indices)

        if cluster_size <= max_cluster_size:
            final_labels[cluster_indices] = label_counter
            label_counter += 1
        else:
            # Split this oversized cluster into chunks of max_cluster_size
            num_subclusters = int(np.ceil(cluster_size / max_cluster_size))
            kmeans = KMeans(n_clusters=num_subclusters, n_init=10, random_state=42)
            sub_labels = kmeans.fit_predict(points[cluster_indices])

            for sub in np.unique(sub_labels):
                sub_indices = cluster_indices[sub_labels == sub]
                final_labels[sub_indices] = label_counter
                label_counter += 1

    return final_labels


cluster_labels = cluster_points(points, cars_available)

c:\Users\Frede\.conda\envs\env_learning_driver_preferences\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
c:\Users\Frede\.conda\envs\env_learning_driver_preferences\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
c:\Users\Frede\.conda\envs\env_learning_driver_preferences\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
c:\Users\Frede\.conda\envs\env_lea

In [33]:
day_df['my_cluster'] =  cluster_labels

In [34]:
day_df.my_cluster.value_counts()

my_cluster
40     437
46     421
61     407
10     379
3      369
      ... 
182      1
239      1
157      1
178      1
134      1
Name: count, Length: 245, dtype: int64